In [76]:
ENDPOINT = "http://rhmobileapps.mtsu.edu:8888/"
DEBUG = False
import requests as rh # rhmobile
import enum
from datetime import datetime
import time

class busColors(enum.Enum):
    RED = 1 
    BLUE = 2
    SILVER = 4 # TODO: Check
    GREEN = 3 # TODO: Check

class busTracker(object):
    def __init__(self):
        self._trackerGood = False
        try:
            response = rh.get(ENDPOINT)
        except Exception as e:
            print(e)
            return
    
        if(response.status_code==200):
            if(DEBUG): print("Tracking available")
            self._trackerGood = True
            self._busDict = {} #map a id to a busTrackingItem
    
    # We aight?
    def getStatus(self):
        return self._trackerGood
    
    def getTime(self):
        now = datetime.now()
        print(now.strftime("%H:%M:%S"))
        return now
    
    # a bit unconventional but GET indicates that its a API call
    def GETBuses(self):
        try:
            response = rh.get(ENDPOINT)
            buses = response.text.split('|')
            array = [x.split(',') for x in buses]

            # type conversion
            for each in array:
                each[0] = int(each[0]) #ID will never be a float
                each[1] = int(each[1]) # bus color
                each[2] = float(each[2]) #
                each[3] = float(each[3])
                temp = busTrackingItem(each[0], each[1], each[2], each[3])
                self.addBus(temp)
            return array
        except Exception as e:
            print(e)
            return -1
    
        
    def addBus(self, busTrackingItem):
        if(busTrackingItem.getID() in self._busDict.keys()):
            if(DEBUG): print("Bus already tracked, will update LAT LON")
        else:
            if(DEBUG): print("Bus not tracked")
        self._busDict.update({busTrackingItem.getID() : busTrackingItem})
    
    # Call this every so often so the dict doesnt get too big.
    # in the programming world, a big dict is not ideal
    def cleanBuses(self):
        toBeCleaned=self.GETBuses()
        fetchedids = [each[0] for each in toBeCleaned]
        storedids = list(self._busDict.keys())
        if(DEBUG): print(fetchedids, storedids)
        
        # if there exists a key in the stored ids that doesn't exist in what we just fetched
        # remove it from the busDict
        for storedid in storedids:
            if storedid in fetchedids: # we still have that bus
                continue
            del self._busDict[storedid]
    
    # really just a alias for get all the buses
    def getBusDict(self):
        return self._busDict
    
    # Fetch specific bus with its ID
    def getBusWithID(self, ID):
        return self._busDict[ID]
    
    # Get tracked buses IDs
    def getTrackedIDs(self):
        return list(self._busDict.keys())
    
class busTrackingItem:
    def __init__(self, id,color, lat, lon):
        self._busID = id
        self._busColor = color
        self._lat = lat
        self._lon = lon
        
    def getID(self):
        return self._busID
    
    def getLatLon(self):
        return self._lat, self._lon
    def printInfo(self):
        print(self.__dict__)

tracker = busTracker()
if(tracker.getStatus()):
    tracker.GETBuses()
    if(DEBUG): print(tracker.getBusDict())
    tracker.cleanBuses()
    cleantime=0
    while(True):
        tracker.GETBuses()
        idlist = tracker.getTrackedIDs()
        tracker.getTime()
        for each in idlist:
                bus = tracker.getBusWithID(each)
                coords = bus.getLatLon()
                bus.printInfo()
                
        print() # segment
        if(cleantime%1000==0):
            print("CLEANING")
            tracker.cleanBuses()
            cleantime = 0
        cleantime+=200
        del idlist
        time.sleep(5) # run this code every n seconds


14:36:24
{'_busID': 1, '_busColor': 1, '_lat': 35.84397673, '_lon': -86.35076999}
{'_busID': 2, '_busColor': 1, '_lat': 35.84905478, '_lon': -86.36410938}
{'_busID': 5, '_busColor': 3, '_lat': 35.85495483, '_lon': -86.37042523}
{'_busID': 6, '_busColor': 1, '_lat': 35.84526252, '_lon': -86.36761232}
{'_busID': 8, '_busColor': 2, '_lat': 35.84520201, '_lon': -86.36678982}
{'_busID': 0, '_busColor': 4, '_lat': 35.84612055, '_lon': -86.37169227}

14:36:29
{'_busID': 1, '_busColor': 1, '_lat': 35.84387015, '_lon': -86.35101239}
{'_busID': 2, '_busColor': 1, '_lat': 35.84902172, '_lon': -86.3638933}
{'_busID': 5, '_busColor': 3, '_lat': 35.85485768, '_lon': -86.36981837}
{'_busID': 6, '_busColor': 1, '_lat': 35.8452943, '_lon': -86.36778528}
{'_busID': 8, '_busColor': 2, '_lat': 35.84521339, '_lon': -86.36691868}
{'_busID': 0, '_busColor': 4, '_lat': 35.84639173, '_lon': -86.37165025}

14:36:34
{'_busID': 1, '_busColor': 1, '_lat': 35.84385232, '_lon': -86.35117966}
{'_busID': 2, '_busColor

KeyboardInterrupt: 